# JAX快速开始

`JAX` 是CPU, GPU和TPU上的Numpy实现，具有出色的自动求导功能，可用于高性能机器学习研究。

使用最新的 [Autograd](https://github.com/hips/autograd) 库，`JAX`可以自动求导原生的Python和Numpy代码。在Python的循环、条件、递归和闭包中也能够轻松使用，甚至可以求微分的微分的微分。它也支持反向模式和正向模式微分，并且以任意的顺序组合。

`JAX`使用 [XLA](https://www.tensorflow.org/xla) 来在GPU或TPU加速器上编译和运行代码。默认情况下编译在后台进行，并且库调用会即时编译（JIT）和执行。`JAX`甚至可以仅用一条函数API来让你将自己写的Python函数即时编译成XLA优化核。您可以任意组合编译和自动微分，无需离开Python即可变大复杂的算法并且获得最佳的性能。

首先我们先导入常用的JAX库：

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

## 矩阵乘法

我们用以下的示例来生成随机数据。与Numpy的一个较大的不同点是，JAX和它生成随机数的方式不同。详细内容参考[JAX锋芒毕露：随机数](https://render.githubusercontent.com/view/ipynb?color_mode=light&commit=6ac9c12ef0d554cbb52e5117d4a87ce431069d39&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f726173696e2d7473756b7562612f4a41585f6368696e6573655f7475746f7269616c2f366163396331326566306435353463626235326535313137643461383763653433313036396433392f636f64652f312e332d4a41582545392539342538422545382538412539322545362541462539352545392539432542322e6970796e62&nwo=rasin-tsukuba%2FJAX_chinese_tutorial&path=code%2F1.3-JAX%E9%94%8B%E8%8A%92%E6%AF%95%E9%9C%B2.ipynb&repository_id=349726397&repository_type=Repository#%F0%9F%94%AA-%E9%9A%8F%E6%9C%BA%E6%95%B0)

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10, ))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


让我们现在开始，给两个大矩阵做乘法：

In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

10.1 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们添加了 `block_until_ready`，因为默认情况下JAX采用异步执行（详见 异步调度）。

JAX的NumPy函数也可以用于普通的NumPy数组。

In [4]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

28.4 ms ± 733 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


这样做会比较慢，因为每次都必须将数据传送到GPU。您可以使用 `device_put()`确保 `NDArray`由设备内存支持。

In [5]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

9.6 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`device_put()` 的输出仍然类似 `NDArray`，但仅需在打印、绘图、保存和分支等需要它们的值的时候才将值复制回CPU。 `device_put()`的行为等效于函数 `jit(lambda x: x)`，但速度更快。

如果你有GPU或TPU，这些调用都会在加速设备上云子那个，并且有比CPU更快的速度。

In [6]:
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit np.dot(x, x.T)

65.7 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


JAX不仅仅是一个由GPU支持的NumPy。他还带有一些程序转换，这些转换在编写数值代码的时候很有用。目前主要有三个：

* `jit()`，用于加速你的代码
* `grad()`，用于微分
* `vmap()`，用于自动向量化或批处理

接下来我们一一介绍，我们还将以有趣的方式来编写这些内容。

## 使用 `jit()`来加速你的代码

JAX可以在GPU（或CPU，如果您没有GPU的话，TPU支持即将到来！）上透明地运行。但是，在上面的实例中，JAX一次只将一个内核分配给GPU操作。如果有一系列操作，则可以使用 `@jit` 装饰器使用XLA一起编译多个操作。让我们尝试一下：

In [7]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000, ))
%timeit selu(x).block_until_ready()

794 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


我们可以用 `@jit`加快速度，它将在首次调用 `selu` 的时候进行jit编译，然后将其缓存。

In [8]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

51.6 µs ± 904 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 使用`grad()`来微分

除了计算数值函数以外，我们还希望对其进行变换。一种变换是[自动微分](https://en.wikipedia.org/wiki/Automatic_differentiation)。在JAX中，就像 [Autograd](https://github.com/HIPS/autograd)，你可以使用`grad()`函数计算微分。

In [9]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661197 0.10499357]


让我们使用差分来验证我们的计算结果是正确的：

In [10]:
def first_finite_differences(f, x):
    eps = 1e-3
    return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps) for v in jnp.eye(len(x))])

print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


求微分就像调用 `grad()`一样容易。`grad()`和`jit()`可以任意组合。在上面的示例中，我们将`sum_logistic`设置为即时编译，然后取其微分。我们也可以更进一步：

In [11]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.035325605


对于更高级的自动微分，可以调用`jax.vjp()`用于反向模式的向量-雅克比积(vector-Jacobian products)和`jax.jvp()`用于前向模式的雅克比-向量积(Jacobian-vector products)。这两者也可以任意组合彼此，也可以与其他的JAX转换互相组合。这里提供了一种他们构成有效计算完整的Hessian矩阵函数的方法：

In [12]:
from jax import jacfwd, jacrev

def hessian(fun):
    return jit(jacfwd(jacrev(fun)))

## 使用`vmap()`来自动向量化

JAX在其API中还有另一种可能有用的转换:`vmap()`，向量化映射。它具有沿数组轴映射函数的类似语义，但不是将循环保留在外部，而是将循环推入函数的原始操作中以提高性能。当与`jit()`组合时，它的速度可以与手动添加批梯度一样快。这里我们将使用一个简单的示例，并使用 `vmap()`将矩阵-向量乘积提升为矩阵-矩阵乘积。尽管在这种特定情况下很容易手动完成此操作，但是这种技术可以用于更加复杂的功能。

In [13]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
    return jnp.dot(mat, v)

给定诸如`apply_matrix()`之类的函数，我们可以在Python中循环执行批处理维度，但是这么做的性能很差。

In [14]:
def naively_batched_apply_matrix(v_batched):
    return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
2.99 ms ± 62.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


我们知道如何手动批处理该操作。在这种情况下，`jnp.dot()`能够透明地处理额外的批处理维度。

In [15]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
34.3 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


然而，假设没有批处理支持，我们的函数可能更加复杂。我们可以使用`vmap()`自动添加批处理支持：

In [16]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

print('Auto-vecctorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vecctorized with vmap
36.8 µs ± 6.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


当然，`vmap()`可以和`jit()`，`grad()`和任何其他JAX变换任意组合。这只是JAX的冰山一角，我们很兴奋您将使用JAX！